In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import os

In [32]:
db_path = os.path.join(os.getcwd(), 'laliga.sqlite')
laliga_con = sqlite3.connect(str(db_path))
cursor = laliga_con.cursor()

In [33]:
cursor.execute('SELECT * FROM Matches')
data = cursor.fetchall()

columns = cursor.execute('PRAGMA table_info(Matches)').fetchall()
columns = [col[1] for col in columns]

df = pd.DataFrame(data, columns=columns)

In [34]:
df = df[df['score'].notna()]
df[['goals_home', 'goals_away']] = df['score'].str.split(':', expand=True).astype(int)
df['diff_goals'] = abs(df['goals_home'] - df['goals_away'])
results = [
    (df['goals_home'] > df['goals_away']),
    (df['goals_home'] == df['goals_away']),
    (df['goals_home'] < df['goals_away'])
]

df['result'] = np.select(results, [1, 0, 2]) # mejor en númeritos, el 0 es el empate
df = df.drop(['score'], axis = 1) # dropeamos el score porque pah k
df.head()


,season,division,matchday,date,time,home_team,away_team,goals_home,goals_away,diff_goals,result
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2,3,1,2
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3,2,1,1
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5,0,5,1
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1,1,0,0
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0,2,2,2


In [10]:
from sklearn.model_selection import train_test_split

for season in df['season'].unique():
    X = df.loc[(df['season'] == season)].drop(['result'], axis = 1) # es la columna que queremos precedir asi que en el dataset de entreno no estará
    y = df.loc[(df['season'] == season)]['result'] # guardamos en una ¿lista? la variable a predecir

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

joderrr
